In [1]:
import pandas as pd
import data_loader as dl

In [12]:
def filter_data(df, cutoff):
    #drop decoy
    df = df[df["decoy"]==False]
    #drop duplicate scans
    df = df.drop_duplicates(subset=["scan"], keep="first") #keep highest coring
    #filter based on qvalue
    df = df[df[df.columns[3]] <= cutoff]
    return df


In [3]:
msgf = dl.clean_msgfplus("2ng")
msgf

,decoy,scan,peptide,QValue
0,False,17836,GVVPLAGTNGETTTQGLDGLSER,0.000000
1,False,14981,LVQDVANNTNEEAGDGTTTATVLAR,0.000000
2,False,12087,AQGPAASAEEPKPVEAPAANSDQTVTVK,0.000000
3,False,12115,SLAGSSGPGASSGTSGDHGELVVR,0.000000
4,False,13798,AIGSASEGAQSSLQEVYHK,0.000000
...,...,...,...,...
16078,True,4676,GGC+57.021GSGC+57.021GGSC+57.021GC+57.021C+57....,0.527173
16079,False,27125,SSQDGSGD,0.527173
16080,True,24458,SSM+15.995SDSAM+15.995,0.527264
16081,True,4504,GGC+57.021GSGC+57.021GGSC+57.021GC+57.021C+57....,0.527356


In [4]:
spec = dl.clean_spectromine("2ng")
spec

,decoy,scan,peptide,PEP.QValue
12545,False,26346,AAAAAAAAAAGAAGGR,0.000000
12546,False,30199,TSLALDESLFR,0.000000
12547,False,13193,ELQNEVEK,0.001448
12548,False,27829,GFAYIEFSDK,0.000000
12549,False,15305,AAAAAAAAAK,0.180843
...,...,...,...,...
53297,True,31195,YFLHEMVYTYNPISR,0.673604
53298,True,15555,YGPEQMWAAGDESNEK,0.482889
53299,True,22740,YSVIGGKK,0.653864
53300,True,26422,YTRPCCPGGHRAGQGQR,0.714074


In [5]:
msf = dl.clean_msfragger("2ng")
msf

,decoy,scan,peptide,PeptideProphet Probability
0,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.10427.10427.2,KLSAPVSQK,0.1059
1,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.10531.10531.4,KSHWDVVRSIVMFGK,0.2933
2,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.10551.10551.3,EDTEEHHLR,0.9912
3,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.10576.10576.2,LSSAMSAAK,0.9973
4,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.10589.10589.3,QVDGDNSHVEMK,0.9816
...,...,...,...,...
13511,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.44019.44019.2,GSDIIVDLEVTLEEVYAGNFVEVVR,1.0000
13512,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.44027.44027.3,GSDIIVDLEVTLEEVYAGNFVEVVR,0.9978
13513,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.44041.44041.3,VGAGAPVYMAAVLEYLTAEILELAGNAAR,1.0000
13514,False,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.44062.44062.3,VGAGAPVYMAAVLEYLTAEILELAGNAAR,0.9989


In [13]:
filter_data(msgf, .01)

,decoy,scan,peptide,QValue
0,False,17836,GVVPLAGTNGETTTQGLDGLSER,0.000000
1,False,14981,LVQDVANNTNEEAGDGTTTATVLAR,0.000000
2,False,12087,AQGPAASAEEPKPVEAPAANSDQTVTVK,0.000000
3,False,12115,SLAGSSGPGASSGTSGDHGELVVR,0.000000
4,False,13798,AIGSASEGAQSSLQEVYHK,0.000000
...,...,...,...,...
4485,False,19347,GAYDIFLNAK,0.009892
4486,False,20413,VPSLVGSFIR,0.009892
4487,False,10287,ATFNPAQDK,0.009892
4488,False,21203,DLYANTVLSGGTTMYPGIADR,0.009892


In [15]:
filter_data(spec, .01)

,decoy,scan,peptide,PEP.QValue
12545,False,26346,AAAAAAAAAAGAAGGR,0.000000
12546,False,30199,TSLALDESLFR,0.000000
12547,False,13193,ELQNEVEK,0.001448
12548,False,27829,GFAYIEFSDK,0.000000
12550,False,37540,VLLESEQFLTELTR,0.000000
...,...,...,...,...
48798,False,14359,YISAAPGAEAK,0.000149
48822,False,29781,YIMVPSGNMGVFDPTEIHNR,0.000000
48841,False,30015,YLVQDTDEFILPTGANK,0.000783
48857,False,15759,YQIDDKPNNQIR,0.000050


In [16]:
filter_data(msf, .01)

,decoy,scan,peptide,PeptideProphet Probability


In [6]:
#look through all the files and get pep and scan #s
files = [".2ng", "2ng"]

file_names = []
prog_type = []
pep =[]
scan = []

for file in files:
    print(file)
    file_names.append(file)
    prog_type.append("spectro")
    df = data_loader.clean_spectromine(file, .01)
    pep.append(df['PEP.StrippedSequence'].nunique())
    scan.append(df['PSM.MS2ScanNumber'].nunique())

    prog_type.append("msgf")
    file_names.append(file)
    df = data_loader.parse_msgfplus(file, .01)
    pep.append(df['new_peptide'].nunique())
    scan.append(df["SpecID"].nunique())
    
    prog_type.append("msfragger")
    file_names.append(file)
    df = data_loader.parse_msfragger(file, .01)
    pep.append(df['Peptide'].nunique())
    scan.append(df['Spectrum'].nunique())

.2ng


NameError: name 'data_loader' is not defined